In [41]:
pip install pulp

In [42]:
import pandas as pd
from pulp import *

In [43]:
# Nom du fichier Excel contenant les données
filename = "data (6).xlsx"

# Importer la première feuille Excel dans un dataframe
aulas = pd.read_excel(filename, sheet_name='aulas')

# Importer la deuxième feuille Excel dans un dataframe
cursos = pd.read_excel(filename, sheet_name='cursos')

# Importer la troisième feuille Excel dans un dataframe
profesores = pd.read_excel(filename, sheet_name='profesors')

# Importer la quatrième feuille Excel dans un dataframe
excepcion = pd.read_excel(filename, sheet_name='excepcion')

# Jours de la semaine
dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']

In [44]:
aulas

,nombre,edificio,capacidad
0,m0.1,edificio1,22
1,m0.2,edificio2,23
2,m0.3,edificio2,20
3,m0.4,edificio1,26
4,m1.1,edificio2,20
5,m1.2,edificio1,18
6,m1.3,edificio2,26
7,m1.4,edificio1,25


In [45]:
cursos

,nombre,idioma,nivel,dias,horario,professor,estudiante
0,curso1,Inglès,A2,"[""Martes""]",14:10-15:50,profesor1,20
1,curso2,Inglès,A1,"[""Lunes"",""Miércoles""]",17:15-18:55,profesor1,26
2,curso3,Inglès,A1,"[""Martes"",""Jueves""]",17:15-18:55,profesor1,24
3,curso4,Inglès,A1,"[""Lunes"",""Miércoles""]",19:00-20:40,profesor1,18
4,curso5,Inglès,A1,"[""Martes"",""Jueves""]",19:00-20:40,profesor1,24
5,curso6,Alemàn,A1,"[""Lunes"",""Miércoles""]",17:15-18:55,profesor2,26
6,curso7,Alemàn,A1,"[""Lunes"",""Miércoles""]",19:00-20:40,profesor2,17
7,curso8,Frances,A2,"[""Lunes""]",12:10-13:50,profesor3,18
8,curso9,Alemàn,A1,"[""Martes"",""Viernes""]",17:15-18:55,profesor2,26
9,curso10,Frances,A1,"[""Lunes"",""Miércoles""]",08:10-09:50,profesor3,17


In [46]:
profesores

,nombre,lugares,idioma
0,profesor1,"[""edificio1"", ""edificio2""]",Inglès
1,profesor2,"[""edificio1""]",Alemàn
2,profesor3,"[""edificio2""]",Frances
3,profesor4,"[""edificio2""]",Espanol


In [47]:
excepcion

,curso,aula
0,curso7,m1.2


In [49]:

  """
    Fonction pour faciliter l'extraction du nom du professeur assurant le cours
    (à partir du nom du cours)

  """
def get_professor(curso_nombre):
    profesor = cursos.loc[cursos['nombre']==curso_nombre]['professor'].item()
    return profesor



  """
    Fonction pour faciliter l'extraction de la liste des bâtiments ou le prof intervient
    (à partir du nom d'un cours assuré par ce dernier)

  """
def get_edificios(curso_nombre):
    edificio_list = eval(profesores.loc[profesores['nombre']==get_professor(curso_nombre)]['lugares'].item())
    return edificio_list

In [89]:
# Création du problème
prob = LpProblem("Optimisation d'attribution des salles de classe", LpMinimize)

# Variables de décision

'''x[c,a] : Variable binaire qui indique si un cours spécifique est assigné à une classe ou pas
  x[c,a] = 1 si le cours c est attribué à la salle a''' 
x = LpVariable.dicts('',[(c,a) for c in cursos['nombre'] for a in aulas['nombre']], lowBound=0, upBound=None, cat='Binary')

''' y[p,a] : Variable binaire qui indique si un prof spécifique utilise une salle spécifique.  
    y[p,a] = 1 si le prof p utilise la salle a, 0 sinon'''
y = LpVariable.dicts('',[(p,a) for p in profesores['nombre'] for a in aulas['nombre']], lowBound=0, upBound=None, cat='Binary')

'''nb_salle : Variable entière qui indique le nombre de salles utilisées par un prof spécifique. '''
nb_salles = LpVariable.dicts('',profesores['nombre'], lowBound=0, cat='LpInteger')

In [90]:
# Fonction-objectif

'''Minimiser le nombre total de salles utilisées par tous les profs'''
prob += lpSum(nb_salles[p] for p in profesores['nombre'])

In [91]:
# Contraintes

'''Contrainte 1 : lie y et x, elle assure que si un cours enseigné par un prof p est assigné à la salle a (x[c,a]=1)
   alors cette salle est utilisée par ce prof (y[p,a]=1)'''
for p in profesores['nombre']:
  for a in aulas['nombre']:
    for c in cursos[cursos['professor']==p]['nombre']:
      prob += y[p,a] >= x[c,a]

'''Contrainte 2 : lie y à nb_salles, nb_salles[p] indique le nombre de salles utilisées par le prof p '''
for p in profesores['nombre']:
  prob += nb_salles[p] == lpSum(y[p,a] for a in aulas['nombre'])

'''Contrainte 3 : assure que chaque cours est obligatoirement assigné à une salle unique'''
for c in cursos['nombre']:
  prob += lpSum(x[c,a] for a in aulas['nombre']) == 1

'''Contrainte 4 : assure que le nombre d'étudiants du cours c ne dépasse pas 
   la capacité de la salle a qui lui a été assignée''' 
for i, c in enumerate(cursos['nombre']):
  for j, a in enumerate(aulas['nombre']):
    prob += x[c,a]*cursos.loc[i,'estudiante'] <= aulas.loc[j,'capacidad']

'''Contrainte 5 : le cours spécifié doit être assigné à la salle spécifiée dans les exceptions'''
for index, row in excepcion.iterrows():
    curso = row['curso']
    aula = row['aula']
    prob += x[curso, aula] == 1

'''Contrainte 6 : pour s'assurer que le prof n'a cours que dans le(s) bâtiment(s) où il intervient'''
for a in aulas['nombre']:
  edificio = aulas.loc[aulas['nombre']==a]['edificio'].item()
  prob += lpSum(x[c,a] for c in cursos['nombre'] if edificio in get_edificios(c)) == lpSum(x[c,a] for c in cursos['nombre'])

'''Contrainte 7 : assure que deux cours ayant lieu le même jour à la même horaire ne soientt pas assigné à la même salle'''
horarios = cursos['horario'].unique()
for a in aulas['nombre']:
  for dia in dias:
      for horaire in horarios:
          cours_meme_horaire = cursos[(cursos['dias'].apply(lambda x: dia in x)) & (cursos['horario'] == horaire)]
          if len(cours_meme_horaire)>1:
              prob += lpSum(x[c,a] for c in cours_meme_horaire['nombre']) <= 1


In [92]:
# Résolution du problème
prob.solve()

# Si la problème est iréalisable, afficher les contraintes à problèmes
if LpStatus[prob.status] != "Optimal":
    for contrainte in prob.constraints.values():
        partie_décimale = value(contrainte)-int(value(contrainte))
        if partie_décimale != 0:
            print(contrainte, "Valeur:", value(contrainte))

# Affichage des résultats
print("Status :", LpStatus[prob.status])
print("nombre total de salles utilisées par tous les profs :", value(prob.objective))

Status : Optimal
nombre total de salles utilisées par tous les profs : 4.0


In [94]:
# Afficher les assignations des cours aux salles
for c, a in x:
  if value(x[c,a])==1:
    print(x[c,a], value(x[c,a]))

# Afficher les salles utilisées par chaque prof
for p, a in y:
  if value(y[p,a])==1:
    print(y[p,a], value(y[p,a]))

_('curso1',_'m1.3') 1.0
_('curso2',_'m1.3') 1.0
_('curso3',_'m1.3') 1.0
_('curso4',_'m1.3') 1.0
_('curso5',_'m1.3') 1.0
_('curso6',_'m0.4') 1.0
_('curso7',_'m1.2') 1.0
_('curso8',_'m1.3') 1.0
_('curso9',_'m0.4') 1.0
_('curso10',_'m1.3') 1.0
_('curso11',_'m1.3') 1.0
_('curso12',_'m1.3') 1.0
_('profesor1',_'m1.3') 1.0
_('profesor2',_'m0.4') 1.0
_('profesor2',_'m1.2') 1.0
_('profesor3',_'m1.3') 1.0


In [239]:
pip install xlsxwriter

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\mpete\OneDrive\Bureau\Upwork\PulP\assignation\Scripts\python.exe -m pip install --upgrade pip' command.


In [240]:
import xlsxwriter

horarios = ["08:10-09:50", "10:10-11:50", "12:10-13:50", "14:10-15:50", "17:15-18:55", "19:00-20:40"]
dias = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes"]

workbook = xlsxwriter.Workbook('emploi_du_temps.xlsx')

for profesor in profesores['nombre']: # Une feuille pour chaque prof
    emploi_du_temps = pd.DataFrame(index=horarios, columns=dias)
    # Remplissage de la grille avec les informations des cours assignés aux salles et professeurs
    for c, a in x:
        if value(x[c,a]) > 0 and cursos.loc[cursos['nombre'] == c, 'professor'].values[0] == profesor:
            horario = cursos.loc[cursos['nombre'] == c, 'horario'].values[0]
            dia = eval(cursos.loc[cursos['nombre'] == c, 'dias'].values[0]) #C'est un Array de jours
            idioma = cursos.loc[cursos['nombre'] == c, 'idioma'].values[0]
            for d in dia:
                emploi_du_temps.loc[horario, d] = f"Cours: {c}\nProfesseur: {profesor}\nLangue: {idioma}"

    # Création du fichier Excel
    worksheet = workbook.add_worksheet(name=profesor) # Nommer la feuille d'après le nom du prof

    # Formatage des cellules
    bold_format = workbook.add_format({'bold': True, 'border':2})
    wrap_format = workbook.add_format({'text_wrap': True, 'border':2})

    # Écriture des en-têtes de colonnes
    for col, dia in enumerate(emploi_du_temps.columns, start=1):
        worksheet.write(0, col, dia, bold_format)

    # Écriture des en-têtes de lignes et des informations des cours
    for row, horario in enumerate(emploi_du_temps.index, start=1):
        worksheet.write(row, 0, horario, bold_format)
        for col, dia in enumerate(emploi_du_temps.columns, start=1):
            info = emploi_du_temps.loc[horario, dia]
            if not isinstance(info,str):
                info = " "
            worksheet.write(row, col, info, wrap_format)

    # Ajustement de la largeur des colonnes
    worksheet.set_column(0, len(dias), 15)

# Enregistrement et fermeture du fichier Excel
workbook.close()